In [1]:
import cv2
from keras.models import load_model
from keras.preprocessing import image
import keras.utils as image
import numpy as np
# import winsound
# frequency = 3000
# duration = 1000
import pyfirmata
import os
import sys
sys.path.append(os.path.abspath("F:\program"))


ModuleNotFoundError: No module named 'pyfirmata'

In [ ]:
new_model = load_model('Drowsyness._detect.h5')

In [ ]:
input_shape = (64,64,1)

In [ ]:
board = pyserial.ArduinoMega('COM3')
buzzer= board.get_pin('d:11:o')
left_indicator = board.get_pin('d:8:o')
right_indicator = board.get_pin('d:9:o')
left_indicator.write(1)
right_indicator.write(1)
buzzer.write(0)
import time

In [ ]:
cap = cv2.VideoCapture(0)
counter = 0

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    # Coordinates of the ROI
    x1 = 270
    y1 = 185
    x2 = 350
    y2 = 215
    # Drawing the ROI
    # The increment/decrement by 1 is to compensate for the bounding box
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    roi = cv2.resize(roi, (input_shape[:2]))
    
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    #ret, roi = cv2.threshold(roi, 95, 255, cv2.THRESH_BINARY)
    y=2
    x_img = image.img_to_array(roi)
    x_img = np.expand_dims(x_img,axis=0)
    y = (new_model.predict(x_img) > 0.5).astype("int32")
    z = new_model.predict(x_img)
    font = cv2.FONT_HERSHEY_SIMPLEX
       
    if y==1 and z==1:
        cv2.putText(frame,text=f'eye open',org = (100,150),fontFace=font, fontScale =1, color=(0,255,0),thickness=3,lineType=cv2.LINE_AA)
        cv2.putText(frame,text=f'{z}',org = (100,200),fontFace=font, fontScale =1, color=(0,255,0),thickness=3,lineType=cv2.LINE_AA)
        buzzer.write(0)
        left_indicator.write(1)
        right_indicator.write(1)
        counter = counter - 1
        if (counter < 0):
            counter = 0
            pass
            
    elif y==0 and z==0:
        cv2.putText(frame,text=f'eye close',org = (100,150),fontFace=font, fontScale =1, color=(0,0,255),thickness=3,lineType=cv2.LINE_AA)
        cv2.putText(frame,text=f'{z}',org = (100,200),fontFace=font, fontScale =1, color=(0,0,255),thickness=3,lineType=cv2.LINE_AA)
        cv2.putText(frame,text=f'counter = '+str(counter),org = (100,250),fontFace=font, fontScale =1, color=(0,0,0),thickness=3,lineType=cv2.LINE_AA)
        
        counter = counter + 1
        if (counter > 30):
            # winsound.Beep(frequency, duration)
            buzzer.write(1)
            left_indicator.write(0)
            right_indicator.write(0)
            time.sleep(0.5)
            buzzer.write(0)
            left_indicator.write(1)
            right_indicator.write(1)
            pass
            
                        
    else:
        cv2.putText(frame,text="no detection",org = (100,150),fontFace=font, fontScale =1, color=(255,255,255),thickness=3,lineType=cv2.LINE_AA)
        left_indicator.write(1)
        right_indicator.write(1)
    
    cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
